In [21]:
from langdetect import detect, DetectorFactory, detect_langs
from deep_translator import GoogleTranslator

# Set seed for consistent language detection results
DetectorFactory.seed = 0

class Translation:
    def __init__(self, text, destination):
        self.text = text
        self.destination = destination
        try:
            detected_languages = detect_langs(self.text)
            print(f"Detected languages with confidence: {detected_languages}")  # Debugging detected languages
            # Use the language with the highest confidence
            self.original = detected_languages[0].lang
            confidence = detected_languages[0].prob
            print(f"Detected language: {self.original} with confidence: {confidence}")
            # Fallback to English if confidence is below 0.8
            if confidence < 0.8:
                print("Low confidence in detected language. Forcing language to English.")
                self.original = "en"
        except Exception as e:
            print(f"Error detecting language: {e}")
            self.original = "en"  # Default to English on error
    def translatef(self):
        try:
            if self.original != self.destination:
                print(f"Translating from {self.original} to {self.destination}: {self.text}")  # Debugging translation
                translator = GoogleTranslator(source=self.original, target=self.destination)
                translation = translator.translate(self.text)
                print(f"Translation result: {translation}")  # Debugging translation result
                return translation
            return self.text  # No translation needed if languages match
        except Exception as e:
            print(f"Error during translation: {e}")
            return self.text  # Return original text if translation fails

In [22]:
import gradio as gr
from load_model import pipe
from utils import Translation

# Add caching for translations and responses
translation_cache = {}
response_cache = {}

def is_greeting(message):
    greetings = ["hello", "hi", "hey"]
    return message.lower() in greetings

def determine_response_length(message):
    # Short responses for greetings or explicit brevity requests
    if is_greeting(message) or "in 10 words" in message.lower():
        return 20  # Very short response
    return 100  # Longer response for detailed questions

def reply(message, history):
    try:
        if message in response_cache:
            print("Using cached response.")  # Debugging
            return response_cache[message]

        # Detect and translate input if needed
        txt = Translation(message, "en")
        print(f"Original language (after fallback if needed): {txt.original}")  # Debugging original language
        if txt.original != "en":
            if message in translation_cache:
                translation = translation_cache[message]
            else:
                translation = txt.translatef()
                translation_cache[message] = translation
        else:
            translation = message

        max_tokens = determine_response_length(translation)
        print(f"Determined max_new_tokens: {max_tokens}")  # Debugging response length

        # Generate response
        guided_message = f"Please provide a concise and relevant response to: {translation}"
        response = pipe(guided_message, max_new_tokens=max_tokens)
        result = response[0]["generated_text"]
        print(f"Generated response: {result}")  # Debugging generated response

        # Translate response back to original language if needed
        if txt.original != "en":
            t = Translation(result, txt.original)
            result = t.translatef()
            print(f"Translated response: {result}")  # Debugging translated response

        response_cache[message] = result
        return result
    except Exception as e:
        print(f"Error in reply function: {e}")
        return "An error occurred. Please try again."

demo = gr.ChatInterface(fn=reply, title="Multilingual-Bloom Bot")
demo.launch(server_name="0.0.0.0", share=False)

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): only one usage of each socket address (protocol/n

* Running on local URL:  http://0.0.0.0:7873

To create a public link, set `share=True` in `launch()`.

To create a public link, set `share=True` in `launch()`.


Original language (after fallback if needed): fi
Determined max_new_tokens: 20


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated response: Please provide a concise and relevant response to: Hello, I am interested in your product. Please send me more information about it.
Hello! My name
Translated response: Anna tiivis ja asiaankuuluva vastaus: Hei, olen kiinnostunut tuotteestasi. Lähetä minulle lisätietoja siitä.
Hei! Minun nimeni
